# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [210]:
import razdel
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter, defaultdict

In [264]:
russian_stopwords = stopwords.words("russian") + ['это', "тебе", 'очень', 'ещё', 'просто', 'вообще']

In [277]:
data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [278]:
data.count()

comment    14412
toxic      14412
dtype: int64

In [279]:
data.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [280]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [281]:
train.head()

,index,comment,toxic
0,9761,"Я всегда с клепками беру, что бы мозг не ебсти...",1.0
1,12685,Только тихо. Первое правило тайной комнаты...\n,0.0
2,12792,А уехать они должны тоже с котом или ко идёт в...,0.0
3,8335,У нас все прыгали исключительно на своих параш...,0.0
4,13067,"сомнительно, что все критично подорожает. Для ...",0.0


In [282]:
vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
X = vectorizer.fit_transform(train.comment) 

In [283]:
def tokenization(s: str):
    tokens = list(razdel.tokenize(s))
    return [_.text for _ in tokens]

cv = TfidfVectorizer(tokenizer=tokenization, stop_words=russian_stopwords)
X_razdel = cv.fit_transform(train.comment)

In [284]:
X_test, X_test_razdel = vectorizer.transform(test.comment), cv.transform(test.comment)

y = train.toxic
y_test = test.toxic

In [285]:
clf = LogisticRegression(C=1, class_weight='unbalanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.78      0.98      0.87       946
         1.0       0.93      0.47      0.62       496

    accuracy                           0.81      1442
   macro avg       0.86      0.72      0.75      1442
weighted avg       0.83      0.81      0.78      1442



In [286]:
clf = LogisticRegression(C=1, class_weight='unbalanced')
clf.fit(X_razdel, y)
preds = clf.predict(X_test_razdel)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.79      0.98      0.87       946
         1.0       0.92      0.51      0.65       496

    accuracy                           0.81      1442
   macro avg       0.85      0.74      0.76      1442
weighted avg       0.83      0.81      0.80      1442



Для логистической регрессии исходя из результатов токенизатор razdel.tokenize немного улучшил результат по f1 метрикам относительно токенизатора по умолчанию TfidfVectorizer. То есть razdel.tokenize отработал лучше.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [222]:
data = [
    "я и ты",
    "ты и я",
    "я, я и только я",
    "только не я",
    "он"
]

# удалим пунктуацию
data = [s.translate(str.maketrans('', '', string.punctuation)) for s in data]

In [223]:
word_index = {word: i for i, word in enumerate(set([word for s in data for word in s.split()]))}
words_count = len([i for s in data for i in s.split()])
N = len(data)

DF = defaultdict(int) 
for word in set(word_index.keys()):
    for text in data:
        words = text.split()
        if word in words:
            DF[word] += 1


In [224]:
tf_idf = [[0 for i in range(len(word_index))] for j in range(len(data))]

for doc in range(len(data)):
    tokens = data[doc].split()
    counter = Counter(tokens)
    for token in np.unique(tokens):
        tf = counter[token] / len(tokens)
        df = DF[token]
        idf = np.log(N/(df))
        tf_idf[doc][word_index[token]] = round(tf*idf, 3)

tf_idf

[[0, 0.074, 0, 0.17, 0, 0.305],
 [0, 0.074, 0, 0.17, 0, 0.305],
 [0.183, 0.134, 0, 0.102, 0, 0],
 [0.305, 0.074, 0.536, 0, 0, 0],
 [0, 0, 0, 0, 1.609, 0]]

In [225]:
df = pd.DataFrame(tf_idf, columns=word_index.keys())
df = df.set_index(pd.Series(data))
df

,только,я,не,и,он,ты
я и ты,0.000,0.074,0.000,0.170,0.000,0.305
ты и я,0.000,0.074,0.000,0.170,0.000,0.305
я я и только я,0.183,0.134,0.000,0.102,0.000,0.000
только не я,0.305,0.074,0.536,0.000,0.000,0.000
он,0.000,0.000,0.000,0.000,1.609,0.000


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [226]:
vectorizer = TfidfVectorizer(min_df=14, max_df=0.37, stop_words=russian_stopwords, max_features=10000, use_idf=True)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

clf = LogisticRegression(C=1, class_weight='unbalanced')
clf.fit(X, y)

preds = clf.predict(X_test)
preds_log = [x[1] for x in clf.predict_proba(X_test)]
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.80      0.96      0.87       962
         1.0       0.86      0.50      0.64       480

    accuracy                           0.81      1442
   macro avg       0.83      0.73      0.75      1442
weighted avg       0.82      0.81      0.79      1442



In [227]:
vectorizer = CountVectorizer(min_df=15, max_df=0.2, max_features=100000, stop_words=russian_stopwords, analyzer="word")
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

# clf = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight='balanced_subsample')
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X, y)

preds = clf.predict(X_test)
preds_nb = [x[1] for x in clf.predict_proba(X_test)]
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.80      0.94      0.86       962
         1.0       0.81      0.53      0.64       480

    accuracy                           0.80      1442
   macro avg       0.81      0.73      0.75      1442
weighted avg       0.80      0.80      0.79      1442



Рассмотрим топ 10 самых токсичных текста из каждого классификатора:

In [228]:
res_nb = zip(preds_nb, test.comment)
res_log = zip(preds_log, test.comment)

res_nb = sorted(res_nb, key=lambda x: x[0], reverse=True)
res_log = sorted(res_log, key=lambda x: x[0], reverse=True)

In [229]:
[i[1] for i in res_log[:10]]

['У-ух, ненавижу, блядь, хохлов.\n',
 'Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n',
 'Ебать вы тупые дебилы, ой блять\n',
 'Опять хохлы из киберсотни на куриную косточку капчуют.\n',
 'Всех же так волнует,что там у хохлов?\n',
 'Хохлы то да разъезжаются ты прав.\n',
 'Какие же хохлы незалежные дегенераты, пиздец просто.\n',
 'если хохлы это лахта,то они существуют страно что еще накрутчик раги не врубили\n',
 'Свин, иди нахуй. Можно хоть один тред без политоты? Даже тебя хуеосить не хочется, давай лучше про няшек-фигуристок говорить.\n',
 'НУ ЧО СУКА ПРОСИЛ СКЕТЧИ? ПРОСИЛ? ПОЛУЧАЙ БЛЯДЬ!!\n']

In [230]:
[i[1] for i in res_nb[:10]]

['Да тупая баба. Видос недавно был: мразь какая то девку у прилавка схватил и говорит отдай деньги, а то я ее зарежу. А ебанутая сука ему говорит Ах перестань хулиганить. И даже когда он телку ножом хуярить начал, она его из ведрышка поливала и шваброй стукнуть пыталась. Пока он ее саму ножом не уебал, до тупой суки не доходило, что при ней человека убивали. Какая слабина, диванные воены? Отдайте все что нужно и вызывайте полицию. Рэмбы комнатные.\n',
 'Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n',
 'Хохлы не люди. Их нужно обязательно бить, унижать, ссать на лицо. Если же пытаться вести с ними хорошо, как это делали в СССР и докрымской России, то они начинают наглеть и безобразничать. Как и все дегенераты, хохлы вежливое отношение принимают за слабость. Увидел хохлы, плюнь ему в рожу и пни под зад сапогом. Ради счастья всего человечества.\n',
 'Какую тему? То, что жиды пидорасы? По-моему это ты пришёл в тред, оскорбился что тут хохлов не обсуждают 

Совпадающие тексты:

In [231]:
set([i[1] for i in res_log[:10]]).intersection(set([i[1] for i in res_nb[:10]]))

{'Ебать вы тупые дебилы, ой блять\n',
 'Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n'}

##### Совпадает всего 2 текста:

'Ебать вы тупые дебилы, ой блять\n'

'Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n'

##### но зато все выделенные тексты можно назвать токсичными. Стоит также заметить, что логистическая регрессия определила самыми токсичными довольно короткие тесты, в отличие от наивного байесовского классификатора.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [265]:
vectorizer = TfidfVectorizer(min_df=17, max_df=0.4, stop_words=russian_stopwords, max_features=10000, use_idf=True)

X = vectorizer.fit_transform(train.comment)
tokens = vectorizer.get_feature_names()

X_test = vectorizer.transform(test.comment)

In [266]:
clf_log = LogisticRegression(C=1, class_weight='unbalanced')
clf_log.fit(X, y)

clf_log_coef = clf_log.coef_

In [267]:
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X, y)

clf_dt_coef = clf_dt.feature_importances_

In [268]:
clf_nb = MultinomialNB()
clf_nb.fit(X, y)

clf_nb_coef = clf_nb.coef_

In [269]:
clf_rf = RandomForestClassifier()
clf_rf.fit(X, y)

clf_rf_coef = clf_rf.feature_importances_

### Рассмотрим топ 5 самых токсичных слов для каждого классификатора:

Логистическая регрессия:

In [270]:
res_log = zip(clf_log_coef[0], tokens)
res_log = sorted(res_log, key=lambda x: x[0], reverse=True)
print([i[1] for i in res_log[:5]])

['хохлы', 'хохлов', 'дебил', 'сука', 'тупые']


Дерево решений:

In [271]:
res_dt = zip(clf_dt_coef, tokens)
res_dt = sorted(res_dt, key=lambda x: x[0], reverse=True)
print([i[1] for i in res_dt[:5]])

['хохлы', 'хохлов', 'нахуй', 'блядь', 'блять']


Наивный байесовский классификатор:

In [272]:
res_nb = zip(clf_nb_coef[0], tokens)
res_nb = sorted(res_nb, key=lambda x: x[0], reverse=True)
print([i[1] for i in res_nb[:5]])

['хохлы', 'почему', 'хохлов', 'нахуй', 'хуй']


Случайный лес:

In [273]:
res_rf = zip(clf_rf_coef, tokens)
res_rf = sorted(res_rf, key=lambda x: x[0], reverse=True)
print([i[1] for i in res_rf[:5]])

['хохлы', 'хохлов', 'нахуй', 'сука', 'хуй']


Можно видеть, что сильными критериями токсичности классификаторы выделяют почти одни и те же слова, которые представляют собой нецензурную лексику. Я дополнила список стоп слов, которых не было в стандартном наборе стоп слов для русского языка nltk (добавила слова, которые на мой взгляд не несут смысловой нагрузки).